In [1]:
import pandas as pd
import ast
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, Dropout, GlobalMaxPool1D, Conv1D, LSTM, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

# Load and clean data
df = pd.read_csv("English_cleaned.csv")

def fix_nested_char_tokens(row):
    if isinstance(row, str):
        row = ast.literal_eval(row)
    flat = []
    for token in row:
        if isinstance(token, list):
            joined = ''.join([c for c in token if c.isalpha()])
            if joined:
                flat.append(joined)
        elif isinstance(token, str):
            cleaned = ''.join(token.split())
            if cleaned:
                flat.append(cleaned)
    return flat

df['tokens'] = df['tokens'].apply(fix_nested_char_tokens)
df['cleanedtext'] = df['tokens'].apply(lambda x: ' '.join(x))
df = df[df['cleanedtext'].str.strip() != '']
df = df.dropna(subset=['cleanedtext'])

# Tokenization and Padding
max_words = 10000
max_len = 100

tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(df['cleanedtext'])
X = tokenizer.texts_to_sequences(df['cleanedtext'])
X = pad_sequences(X, maxlen=max_len)

# Labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['target'])  # -1 → 0, 1 → 1
y_cat = to_categorical(y)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y_cat, test_size=0.2, random_state=42, stratify=y_cat)

early_stop = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

# ========== Model 1: Dense Feedforward ==========
def create_dense_model():
    model = Sequential([
        Embedding(input_dim=max_words, output_dim=64, input_length=max_len),
        GlobalMaxPool1D(),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(32, activation='relu'),
        Dropout(0.3),
        Dense(2, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# ========== Model 2: 1D CNN ==========
def create_cnn_model():
    model = Sequential([
        Embedding(input_dim=max_words, output_dim=64, input_length=max_len),
        Conv1D(128, 5, activation='relu'),
        GlobalMaxPool1D(),
        Dropout(0.5),
        Dense(64, activation='relu'),
        Dense(2, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# ========== Model 3: Bidirectional LSTM ==========
def create_bilstm_model():
    model = Sequential([
        Embedding(input_dim=max_words, output_dim=64, input_length=max_len),
        Bidirectional(LSTM(64, return_sequences=False)),
        Dropout(0.5),
        Dense(32, activation='relu'),
        Dense(2, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# ========== Training and Evaluation Function ==========
def train_and_evaluate(model_fn, name):
    print(f"\nTraining {name}...")
    model = model_fn()
    model.fit(X_train, y_train, epochs=5, batch_size=128, validation_split=0.1, callbacks=[early_stop], verbose=2)

    y_pred_probs = model.predict(X_test)
    y_pred = np.argmax(y_pred_probs, axis=1)
    y_true = np.argmax(y_test, axis=1)

    acc = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, average='weighted')
    print(f"{name} Accuracy: {acc:.4f}")
    print(f"{name} F1 Score: {f1:.4f}")
    return model, acc, f1

# ========== Run All Models ==========
results = []
models = [
    ("Dense Feedforward", create_dense_model),
    ("1D CNN", create_cnn_model),
    ("BiLSTM", create_bilstm_model)
]

for name, fn in models:
    model, acc, f1 = train_and_evaluate(fn, name)
    results.append((name, acc, f1))

# ========== Print Summary ==========
print("\n\n=== Results Summary ===")
for name, acc, f1 in sorted(results, key=lambda x: x[1], reverse=True):
    print(f"{name}: Accuracy = {acc:.4f}, F1 = {f1:.4f}")


Training Dense Feedforward...


e:\Social Media Sentiment Analysis\venv\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/5
7813/7813 - 124s - 16ms/step - accuracy: 0.7660 - loss: 0.4900 - val_accuracy: 0.7769 - val_loss: 0.4677
Epoch 2/5
7813/7813 - 151s - 19ms/step - accuracy: 0.7837 - loss: 0.4621 - val_accuracy: 0.7791 - val_loss: 0.4639
Epoch 3/5
7813/7813 - 152s - 19ms/step - accuracy: 0.7912 - loss: 0.4485 - val_accuracy: 0.7794 - val_loss: 0.4635
Epoch 4/5
7813/7813 - 149s - 19ms/step - accuracy: 0.7983 - loss: 0.4367 - val_accuracy: 0.7791 - val_loss: 0.4657
Epoch 5/5
7813/7813 - 189s - 24ms/step - accuracy: 0.8043 - loss: 0.4264 - val_accuracy: 0.7791 - val_loss: 0.4670
8681/8681 ━━━━━━━━━━━━━━━━━━━━ 15s 2ms/step
Dense Feedforward Accuracy: 0.7795
Dense Feedforward F1 Score: 0.7794

Training 1D CNN...


e:\Social Media Sentiment Analysis\venv\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/5
7813/7813 - 661s - 85ms/step - accuracy: 0.7675 - loss: 0.4832 - val_accuracy: 0.7791 - val_loss: 0.4652
Epoch 2/5
7813/7813 - 674s - 86ms/step - accuracy: 0.7851 - loss: 0.4560 - val_accuracy: 0.7842 - val_loss: 0.4625
Epoch 3/5
7813/7813 - 691s - 88ms/step - accuracy: 0.7926 - loss: 0.4425 - val_accuracy: 0.7839 - val_loss: 0.4609
Epoch 4/5
7813/7813 - 656s - 84ms/step - accuracy: 0.7998 - loss: 0.4306 - val_accuracy: 0.7838 - val_loss: 0.4595
Epoch 5/5
7813/7813 - 647s - 83ms/step - accuracy: 0.8060 - loss: 0.4197 - val_accuracy: 0.7818 - val_loss: 0.4640
8681/8681 ━━━━━━━━━━━━━━━━━━━━ 68s 8ms/step
1D CNN Accuracy: 0.7835
1D CNN F1 Score: 0.7835

Training BiLSTM...


e:\Social Media Sentiment Analysis\venv\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/5
7813/7813 - 2281s - 292ms/step - accuracy: 0.7720 - loss: 0.4764 - val_accuracy: 0.7834 - val_loss: 0.4576
Epoch 2/5
7813/7813 - 2218s - 284ms/step - accuracy: 0.7897 - loss: 0.4489 - val_accuracy: 0.7877 - val_loss: 0.4502
Epoch 3/5
7813/7813 - 2051s - 263ms/step - accuracy: 0.7974 - loss: 0.4355 - val_accuracy: 0.7882 - val_loss: 0.4501
Epoch 4/5
7813/7813 - 2057s - 263ms/step - accuracy: 0.8038 - loss: 0.4241 - val_accuracy: 0.7885 - val_loss: 0.4505
Epoch 5/5
7813/7813 - 2050s - 262ms/step - accuracy: 0.8099 - loss: 0.4134 - val_accuracy: 0.7874 - val_loss: 0.4547
8681/8681 ━━━━━━━━━━━━━━━━━━━━ 280s 32ms/step
BiLSTM Accuracy: 0.7887
BiLSTM F1 Score: 0.7886


=== Results Summary ===
BiLSTM: Accuracy = 0.7887, F1 = 0.7886
1D CNN: Accuracy = 0.7835, F1 = 0.7835
Dense Feedforward: Accuracy = 0.7795, F1 = 0.7794


In [2]:
bilstm_model = model  # 'model' refers to the last trained model
bilstm_model.save("bilstm_model.keras")

In [29]:
def predict_sample_text(sample_text, model, tokenizer, label_encoder, threshold=0.2):
    # Preprocess input
    sequence = tokenizer.texts_to_sequences([sample_text])
    padded = pad_sequences(sequence, maxlen=max_len)

    # Predict
    pred_prob = model.predict(padded)
    pred_class = np.argmax(pred_prob, axis=1)[0]
    original_label = label_encoder.inverse_transform([pred_class])[0]

    # Calculate confidence gap between the top two probabilities
    sorted_probs = sorted(pred_prob[0], reverse=True)
    confidence_gap = sorted_probs[0] - sorted_probs[1]

    # Map to string labels with neutral for close probabilities
    if confidence_gap < threshold:
        sentiment = "neutral"
    else:
        sentiment = "positive" if original_label == 1 else "negative"

    # Print the results
    print(f"Input Text: {sample_text}")
    print(f"Predicted Sentiment: {sentiment}")
    print(f"Predicted Probabilities: {pred_prob[0]}")  # Show the probabilities for each class

In [26]:
predict_sample_text("I love this product, it's amazing!", bilstm_model, tokenizer, label_encoder)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
Input Text: I love this product, it's amazing!
Predicted Sentiment: positive
Predicted Probabilities: [0.13570344 0.86429656]


In [27]:
test_samples = [
    "I love this product!",
    "This is the worst experience I've ever had.",
    "Absolutely fantastic performance.",
    "Not what I expected, pretty bad.",
    "It was okay, not great but not terrible.",
    "I'm extremely satisfied with the service.",
    "Terrible! I want my money back.",
    "Such a pleasant surprise.",
    "Disappointing result after all the hype.",
    "Highly recommend this to everyone!"
]

for text in test_samples:
    predict_sample_text(text, bilstm_model, tokenizer, label_encoder)
    print("-" * 50)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
Input Text: I love this product!
Predicted Sentiment: positive
Predicted Probabilities: [0.07155262 0.9284473 ]
--------------------------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
Input Text: This is the worst experience I've ever had.
Predicted Sentiment: negative
Predicted Probabilities: [0.7588108  0.24118914]
--------------------------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Input Text: Absolutely fantastic performance.
Predicted Sentiment: positive
Predicted Probabilities: [0.3228619 0.6771381]
--------------------------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
Input Text: Not what I expected, pretty bad.
Predicted Sentiment: negative
Predicted Probabilities: [0.7433778  0.25662217]
--------------------------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
Input Text: It was okay, not great but not terrible.
Predicted Sentiment: positive
Predicted Probabilities: [0.1

In [30]:
test_samples = [
    "This movie was a masterpiece, I loved every second of it!",
    "I can't believe how bad the food was. It was cold and tasteless.",
    "The event was well organized and enjoyable. I had a great time.",
    "I’m really disappointed with the quality of this product. It broke after one use.",
    "The hotel was nice, but the staff could have been more helpful.",
    "It’s a decent book, but the ending didn’t live up to the expectations.",
    "I had such an amazing time at the concert. Highly recommended!",
    "I’m never buying from this store again. Terrible customer service.",
    "The app is okay but keeps crashing. Needs improvement.",
    "Such a lovely day at the park, the weather was perfect!"
]

for text in test_samples:
    predict_sample_text(text, bilstm_model, tokenizer, label_encoder)
    print("-" * 50)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
Input Text: This movie was a masterpiece, I loved every second of it!
Predicted Sentiment: positive
Predicted Probabilities: [0.31940058 0.6805994 ]
--------------------------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
Input Text: I can't believe how bad the food was. It was cold and tasteless.
Predicted Sentiment: negative
Predicted Probabilities: [0.63131195 0.36868805]
--------------------------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
Input Text: The event was well organized and enjoyable. I had a great time.
Predicted Sentiment: positive
Predicted Probabilities: [0.09833427 0.9016658 ]
--------------------------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Input Text: I’m really disappointed with the quality of this product. It broke after one use.
Predicted Sentiment: negative
Predicted Probabilities: [0.72360665 0.2763934 ]
--------------------------------------------------
1/1 ━━━━━━━

In [32]:
neutral_test_samples = [
    "The meeting started at 10 AM and ended on time.",
    "I received the package yesterday.",
    "It's just another regular day.",
    "The results were as expected.",
    "I tried the product; it works as described.",
    "This version is different, but I’m still getting used to it.",
    "There was traffic on the way home, like usual.",
    "The movie was neither good nor bad, just average.",
    "I used the app. It functions like most others.",
    "Not much to say about this experience.",
    "The service was acceptable, nothing special.",
    "The product came in standard packaging.",
    "The presentation covered all the required topics.",
    "The quality is fine for the price.",
    "It looks okay. Nothing stands out.",
]

for text in neutral_test_samples:
    predict_sample_text(text, bilstm_model, tokenizer, label_encoder)
    print("-" * 50)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
Input Text: The meeting started at 10 AM and ended on time.
Predicted Sentiment: positive
Predicted Probabilities: [0.29524964 0.70475036]
--------------------------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
Input Text: I received the package yesterday.
Predicted Sentiment: neutral
Predicted Probabilities: [0.43696126 0.56303877]
--------------------------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
Input Text: It's just another regular day.
Predicted Sentiment: neutral
Predicted Probabilities: [0.40240726 0.59759265]
--------------------------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
Input Text: The results were as expected.
Predicted Sentiment: positive
Predicted Probabilities: [0.3144089  0.68559116]
--------------------------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
Input Text: I tried the product; it works as described.
Predicted Sentiment: positive
Predicted Proba